In [1]:
%load_ext autoreload
%autoreload 2

In [120]:
import requests
from session import SESSION
from common import print_problem, get_problem_input, submit_answer, neighbors
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from collections import Counter, namedtuple
import re
from collections import defaultdict
from copy import copy, deepcopy
import itertools
from itertools import accumulate, zip_longest, pairwise, tee
from pprint import pprint
from functools import cache
import numpy as np
from enum import Enum, IntEnum
from string import ascii_lowercase, ascii_uppercase
from dataclasses import dataclass, field
import operator
from typing import List, Tuple, Set, Dict
import functools
np.set_printoptions(linewidth=160)

In [121]:
DAY = 16

In [4]:
raw_data, data = get_problem_input(DAY)

Raw Data:
'Valve HM has flow rate=0; tunnels lead to valves LS, YS\nValve IY has flow rate=1'
Split Data:
['Valve HM has flow rate=0; tunnels lead to valves LS, YS',
 'Valve IY has flow rate=15; tunnels lead to valves YI, MU, KN, QS, QM',
 'Valve VI has flow rate=22; tunnels lead to valves LE, SE, RB, JR',
 'Valve SE has flow rate=0; tunnels lead to valves VI, AZ',
 'Valve QU has flow rate=0; tunnels lead to valves YC, QK',
 'Valve RB has flow rate=0; tunnels lead to valves AN, VI',
 'Valve PU has flow rate=0; tunnels lead to valves JR, IM',
 'Valve OA has flow rate=0; tunnels lead to valves KZ, FR',
 'Valve AQ has flow rate=23; tunnels lead to valves FA, QM, GE',
 'Valve QS has flow rate=0; tunnels lead to valves IM, IY']


In [122]:
test_data = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II""".split("\n")

In [124]:
line_re = re.compile(r"^Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.+)")

In [125]:
@dataclass
class Valve:
    name: str
    flow: int
    outlets: List[str]
    is_open: bool = field(init=False, default=False)

In [126]:
def parse(str_data):
    ret = {}
    for l in str_data:
        name, flow, outlets = line_re.match(l).groups()
        outlets = outlets.replace(" ","").split(",")
        ret[name] = Valve(name, int(flow), outlets)
    return ret

In [127]:
parse(test_data)

{'AA': Valve(name='AA', flow=0, outlets=['DD', 'II', 'BB'], is_open=False),
 'BB': Valve(name='BB', flow=13, outlets=['CC', 'AA'], is_open=False),
 'CC': Valve(name='CC', flow=2, outlets=['DD', 'BB'], is_open=False),
 'DD': Valve(name='DD', flow=20, outlets=['CC', 'AA', 'EE'], is_open=False),
 'EE': Valve(name='EE', flow=3, outlets=['FF', 'DD'], is_open=False),
 'FF': Valve(name='FF', flow=0, outlets=['EE', 'GG'], is_open=False),
 'GG': Valve(name='GG', flow=0, outlets=['FF', 'HH'], is_open=False),
 'HH': Valve(name='HH', flow=22, outlets=['GG'], is_open=False),
 'II': Valve(name='II', flow=0, outlets=['AA', 'JJ'], is_open=False),
 'JJ': Valve(name='JJ', flow=21, outlets=['II'], is_open=False)}

In [164]:
def open_valves(valve_data: Dict[str, Valve], remaining_time: int):
    at_valve:str = "AA"
    in_transit:bool=False
    open_valves = dict()
    for remain in range(1, remaining_time+1):
        print(f"== Minute {remain} ==")
        if not open_valves:
            print("No valves are open.")
        v = valve_data[at_valve]
        if not in_transit:
            possibles = [valve_data[x] for x in v.outlets]
            highest_flow = sorted(possibles, key=lambda y: (y.is_open, -y.flow))
            print(highest_flow)
            target = highest_flow[0]
            at_valve = target.name
            in_transit=True
            print(f"You move to valve {at_valve}.")
        else:
            in_transit=False
            open_valves[v.name] = v.flow
            v = valve_data[at_valve].is_open = True
            print(f"You open valve {at_valve}.")

In [165]:
open_valves(parse(test_data), 30)

== Minute 1 ==
No valves are open.
[Valve(name='DD', flow=20, outlets=['CC', 'AA', 'EE'], is_open=False), Valve(name='BB', flow=13, outlets=['CC', 'AA'], is_open=False), Valve(name='II', flow=0, outlets=['AA', 'JJ'], is_open=False)]
You move to valve DD.
== Minute 2 ==
No valves are open.
You open valve DD.
== Minute 3 ==
[Valve(name='EE', flow=3, outlets=['FF', 'DD'], is_open=False), Valve(name='CC', flow=2, outlets=['DD', 'BB'], is_open=False), Valve(name='AA', flow=0, outlets=['DD', 'II', 'BB'], is_open=False)]
You move to valve EE.
== Minute 4 ==
You open valve EE.
== Minute 5 ==
[Valve(name='FF', flow=0, outlets=['EE', 'GG'], is_open=False), Valve(name='DD', flow=20, outlets=['CC', 'AA', 'EE'], is_open=True)]
You move to valve FF.
== Minute 6 ==
You open valve FF.
== Minute 7 ==
[Valve(name='GG', flow=0, outlets=['FF', 'HH'], is_open=False), Valve(name='EE', flow=3, outlets=['FF', 'DD'], is_open=True)]
You move to valve GG.
== Minute 8 ==
You open valve GG.
== Minute 9 ==
[Valve(n

In [18]:
submit_answer(DAY, 2, Out[max(Out.keys())])

27625
("That's the right answer!  You are one gold star closer to collecting enough "
 'star fruit.You have completed Day 14! You can [Shareon\n'
 '  Twitter\n'
 'Mastodon] this victory or [Return to Your Advent Calendar].')


True